# setup

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity


HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata


BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [2]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# vision

In [3]:
class VDataset(Dataset):
    def __init__(self, df, p_imgs, transforms):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_id = self.df.image[idx]
        img_path = f"{p_imgs}/{img_id}"
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

# https://github.com/lukemelas/EfficientNet-PyTorch
tfms = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

ds = VDataset(df, p_imgs, tfms)

dl = DataLoader(dataset=ds, 
                batch_size=BS,
                num_workers=NWKRS,
                pin_memory=PIN_MEMORY,
                shuffle=False)


device = torch.device(DEVICE)
mdlv = EfficientNet.from_pretrained('efficientnet-b0').to(device)
mdlv.eval()

feats = np.zeros((len(ds), 1280))
i = 0
for dat in tqdm(dl):
    with torch.no_grad():
        fts = mdlv.extract_features(dat.to(device)).mean(dim=(-1,-2))
    l = len(fts)
    feats[i:i+l,:] = fts.cpu().detach().numpy()
    i+=l

Loaded pretrained weights for efficientnet-b0


100%|██████████| 134/134 [01:54<00:00,  1.17it/s]


In [4]:
df.shape, feats.shape

((34250, 5), (34250, 1280))

In [5]:
del dl, ds, dat, fts, mdlv

In [6]:
gc.collect()
torch.cuda.empty_cache()

In [7]:
vfeats = feats

# text

In [8]:
def mk_tensors(txt, tokenizer, maxlen):
    tok_res = tokenizer(
        txt, truncation=True, padding=True, max_length=maxlen
    )
    input_ids = tok_res["input_ids"]
    token_type_ids = tok_res["token_type_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    token_type_ids = torch.tensor(token_type_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask, token_type_ids


def mk_dl(tensors):
    input_ids, token_type_ids, attention_mask = tensors
    ds = TensorDataset(input_ids, attention_mask, token_type_ids)
    dl = DataLoader(dataset=ds, 
                batch_size=BS,
                num_workers=NWKRS,
                pin_memory=PIN_MEMORY,
                shuffle=False)
    return dl

In [9]:
# 'bert-base-multilingual-cased',
# 'xlm-roberta-base',
# 'pvl/labse_bert'
nm_mdl = 'pvl/labse_bert'
tokenizer = AutoTokenizer.from_pretrained(nm_mdl, do_lower_case=False)

tensors = mk_tensors(list(df.title.values), tokenizer, maxlen=MAXLEN)

dl = mk_dl(tensors)

mdlt = AutoModel.from_pretrained(nm_mdl).to(device)
mdlt.eval()

feats = np.zeros((len(df), 768))
i = 0
for dat in tqdm(dl):
    with torch.no_grad():
        dat = (o.to(device) for o in dat)
        output = mdlt(*dat)
        fts = output.last_hidden_state
        fts = fts.mean(dim=-2)
        l = len(fts)
        feats[i:i+l,:] = fts.cpu().detach().numpy()
        i+=l

100%|██████████| 134/134 [01:13<00:00,  1.82it/s]


In [10]:
del dl, dat, fts,mdlt

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [12]:
df.shape, feats.shape

((34250, 5), (34250, 768))

In [13]:
tfeats = feats

# hash

# similarity

In [14]:
%%time
vsims = cosine_similarity(vfeats)

CPU times: user 53.2 s, sys: 4.83 s, total: 58.1 s
Wall time: 58 s


In [15]:
%%time
tsims = cosine_similarity(tfeats)

CPU times: user 37.6 s, sys: 4.86 s, total: 42.4 s
Wall time: 42.4 s


In [25]:
vsims.shape, tsims.shape

((34250, 34250), (34250, 34250))

In [42]:
i = np.random.choice(range(len(df)))
print(i)
df.iloc[i].to_dict()

14578


{'posting_id': 'train_1705672160',
 'image': '6dbe6655134dd28949866dbba1c5a278.jpg',
 'image_phash': 'a3f3710e1ccc58a7',
 'title': '[BPOM] Avoskin Perfect Hydrating Treatment Essence 30ml',
 'label_group': 606611277}

In [48]:
grp = df.iloc[i].label_group

df[df.label_group==grp][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
11084,AVOSKIN Perfect Hydrating Treatment Essence 30...,b7f571481ccc5a07,606611277
14578,[BPOM] Avoskin Perfect Hydrating Treatment Ess...,a3f3710e1ccc58a7,606611277
18215,Avoskin PHTE 30 ml (perfect hydrating treatmen...,ba33c00cccb772e3,606611277
20965,AVOSKIN PERFECT HYDRATNG TREATMENT ESSENCE (PHTE),d78e2459993159ae,606611277
33139,Avoskin Perfect Hydrating Treatment Essence 30ml,a3f3710e1ccc58a7,606611277


In [49]:
ntop = 5

In [50]:
idx_sim = np.argsort(-tsims[i])[:ntop]
df.iloc[idx_sim][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
14578,[BPOM] Avoskin Perfect Hydrating Treatment Ess...,a3f3710e1ccc58a7,606611277
23826,[BPOM] AVOSKIN Perfect Hydrating Treatment Ess...,fe0c6c7181c3c3e3,1484811081
33139,Avoskin Perfect Hydrating Treatment Essence 30ml,a3f3710e1ccc58a7,606611277
25191,(BPOM) AVOSKIN Perfect Hydrating Treatment Ess...,b0f803cec687ff04,1484811081
794,Avoskin PHTE Perfect Hydrating Treatment Essen...,fe0c6c7181c3c3e3,1484811081


In [51]:
idx_sim = np.argsort(-vsims[i])[:ntop]
df.iloc[idx_sim][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
14578,[BPOM] Avoskin Perfect Hydrating Treatment Ess...,a3f3710e1ccc58a7,606611277
33139,Avoskin Perfect Hydrating Treatment Essence 30ml,a3f3710e1ccc58a7,606611277
11084,AVOSKIN Perfect Hydrating Treatment Essence 30...,b7f571481ccc5a07,606611277
3424,Avoskin PHTE Perfect Hydrating Treatment Essen...,e73333cccccc9889,1484811081
10477,Avoskin Perfect Hydrating Treatment Essence (P...,e666268b99999999,2952860307


# save 

In [34]:
pout = f"{HOME}/data/dev0005"
!mkdir -p $pout

In [35]:
np.save(f"{pout}/vfeats.npy", vfeats)
np.save(f"{pout}/tfeats.npy", tfeats)

np.save(f"{pout}/vsims.npy", vsims)
np.save(f"{pout}/tsims.npy", tsims)